In [1]:
import pandas as pd
import numpy as np
from metrics_forecast_class import TimeSeriesEvaluator,MultiSeriesEvaluator
h=6
window=24
df= pd.read_csv('~/rug/thesis/data/influencer_sample010525.csv')
df['published_date'] = pd.to_datetime(df['publishedDate'])
df
df_temp= df.query('channel_uid=="51b52b775df43f1ea0abf5db8789446d"')[['published_date', 'video_plays']].dropna()
df_temp = df_temp.set_index('published_date').sort_index()
y= df_temp['video_plays']
y = y.sort_index()
y_pos = y.reset_index(drop=True)

#y = y.asfreq('D') 
y

published_date
2021-09-09 19:23:35+00:00    4187.0
2021-09-16 12:17:06+00:00    3216.0
2021-09-21 12:15:06+00:00    1914.0
2021-09-22 11:24:18+00:00     656.0
2021-09-25 19:34:55+00:00    1619.0
                              ...  
2024-12-27 14:01:46+00:00    2017.0
2025-01-07 13:34:13+00:00    2425.0
2025-01-10 10:32:02+00:00    1828.0
2025-01-31 22:21:19+00:00    1060.0
2025-02-19 14:56:59+00:00    1550.0
Name: video_plays, Length: 249, dtype: float64

evaluator = TimeSeriesEvaluator(y=y, initial_window=12, step_length=6, h=6)
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.auto_reg import AutoREG
from sktime.forecasting.fbprophet import Prophet

evaluator.evaluate_sktime_model("AutoARIMA", AutoARIMA())
evaluator.evaluate_sktime_model("Theta", ThetaForecaster(sp=1))
evaluator.evaluate_sktime_model("ETS", AutoETS(sp=1))
evaluator.evaluate_sktime_model("Prophet", Prophet(
    seasonality_mode='multiplicative',
    n_changepoints=int(len(y) / 12)))
# Evaluate AR models with various lags
for lag in [2, 3, 4, 5]:
    evaluator.evaluate_sktime_model(f"AutoREG({lag})", AutoREG(lags=lag))

# --- Print results ---
summary_df = evaluator.get_results_df()


In [2]:
df.groupby('channel_uid').count()

,post_uid,publishedDate,metrics,potentialReach,source,comments,video_views,engagements,video_plays,shares,published_date
channel_uid,,,,,,,,,,,
004621c59168375c92abc5072b0e9448,801,801,778,801,801,778,41,778,8,0,801
03bbf13a212736adaa05fde51f363d5e,37,37,37,37,37,37,37,33,37,37,37
05fe8678b3c432c1834a8249b9b5fc2a,191,191,191,191,191,191,128,185,139,6,191
0811a952ce6636fdb283cca1fd6b601a,1170,1170,1170,1170,1170,1170,1170,1170,1170,1170,1170
08c242dca64f305598abbd56e7b22c58,305,305,305,305,305,305,81,281,47,24,305
...,...,...,...,...,...,...,...,...,...,...,...
f7674e4b4d7d3e3c90c7490028b5345a,1479,1479,760,1479,1479,760,221,705,38,55,1479
f8348ae02f753e0da6fe50837049b4d4,20,20,20,20,20,20,20,20,20,20,20
f84b3aaef6583cb9bd11ec8a859e5810,217,217,217,217,217,217,217,217,217,217,217


In [3]:
ids_test=df.query("""source in ['instagram-content','youtube-content','tiktok-content','facebook-content']""")\
    ['channel_uid'].drop_duplicates().sample(20, random_state=1).values
df_temp=df.query('channel_uid in @ids_test')
df_temp['source'].unique()

array(['tiktok-content', 'tiktok-content-oauth', 'instagram-content',
       'facebook-content', 'youtube-content',
       'instagram-stories-social-data', 'instagram-reels',
       'instagram-igtv', 'tiktok-content-public'], dtype=object)

In [4]:
multi_eval = MultiSeriesEvaluator(df_temp, date_col='published_date', channel_col='channel_uid',metric_cols=['video_plays','engagements'])
multi_eval.run(h=6, initial_window=24, step_length=6)

summary_df = multi_eval.get_summary()
print(summary_df)

Evaluating models:   0%|          | 0/40 [00:00<?, ?it/s]/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Evaluating channel: e4786c9ac7d23a40aa6840e8d817642b
Evaluating metric: video_plays


Importing plotly failed. Interactive plots will not work.


Evaluating AutoARIMA for e4786c9ac7d23a40aa6840e8d817642b/video_plays


/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: '

KeyboardInterrupt: 

In [9]:
import datetime as dt
summary_df.to_csv(f'../cache/summary_df_{dt.datetime.now():%y%m%d%H%M}.csv', index=False)
summary_df

,model_name,test_MeanAbsoluteError,test_MeanAbsolutePercentageError,test_MeanSquaredError,channel_uid,metric_name,source
0,AutoARIMA,0.000005,2.172788e+10,0.000005,f84b3aaef6583cb9bd11ec8a859e5810,video_plays,youtube-content
1,ETS,0.000000,0.000000e+00,0.000000,f84b3aaef6583cb9bd11ec8a859e5810,video_plays,youtube-content
2,Prophet,0.000000,0.000000e+00,0.000000,f84b3aaef6583cb9bd11ec8a859e5810,video_plays,youtube-content
3,AutoREG(2),0.000000,0.000000e+00,0.000000,f84b3aaef6583cb9bd11ec8a859e5810,video_plays,youtube-content
4,AutoREG(3),0.000000,0.000000e+00,0.000000,f84b3aaef6583cb9bd11ec8a859e5810,video_plays,youtube-content
5,AutoREG(4),0.000000,0.000000e+00,0.000000,f84b3aaef6583cb9bd11ec8a859e5810,video_plays,youtube-content
6,AutoREG(5),0.000000,0.000000e+00,0.000000,f84b3aaef6583cb9bd11ec8a859e5810,video_plays,youtube-content
7,AutoARIMA,1773.312844,5.842613e-01,2345.515395,f84b3aaef6583cb9bd11ec8a859e5810,engagements,youtube-content
8,Theta,1787.474102,6.226157e-01,2330.373080,f84b3aaef6583cb9bd11ec8a859e5810,engagements,youtube-content
9,ETS,1738.316593,5.663295e-01,2334.973413,f84b3aaef6583cb9bd11ec8a859e5810,engagements,youtube-content
